
## Notebook made by  

|** Name** | **Student id** | **email**|
|:- |:-|:-|
|Dylan Wijman |10651012|dylanwijman@hotmail.com|
|Eline Steensma|10589813|elinesteensma@gmail.com|
|Sjoerd Paardekooper|10278397|sjoerd.paardekooper@gmail.com|

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )


Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. 

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img src='https://github.com/dylanwijman/data-science/blob/master/Afbeeldingen/selfies.jpg?raw=true'/>


### Note
* **Assignments without the selfies or completely filled in information will not be graded and receive 0 points.**


# Part 1: obtaining information from the web

### RSS parsing

Make a notebook that performs the following steps.

1. Create a script that retrieves all urls of rss feeds from <http://www.volkskrant.nl/rss/feeds/>. Use urllib2 and beautifulsoup for this. Store the urls in a list.
    * **update 2016**
    * As all Dutch sites, Volkskrant asks whether you accept cookies. This makes simple collecting webpages a lot harder. 
    * The code in the code cell below does the trick. 
    * After running this, I could collect further files from Volkskrant without additional cookie hassle.
2. Download all rss feeds and store them on disk.
3. Parse all RSS feeds using `lxml`. Create a list of  dicts with fields "channel", "url", "title", "date" in which you store this information for each item.
4. Compute some statistics about this dict: how many items, how many per channel, are there doubles (items occuring in several channels), etc.
5. Write this list as a csv file, store on disk, and upload to Google fusion tables.
6. Download all articles (once), parse out the text and store as pairs (date,text) in a list.
7. Count per day the number of words, and the number of unique words. Show this in a plot.

In [1]:
import cookielib # Thanks to http://stackoverflow.com/questions/29395407/enabling-cookies-with-urllib
import urllib2
import urllib
import json
import requests
import os
from bs4 import BeautifulSoup
import re 
import feedparser
from lxml import etree
from lxml import objectify

url = 'http://www.volkskrant.nl/rss/feeds/'

# with urllib2 and handling cookies
cookiejar= cookielib.LWPCookieJar()
opener= urllib2.build_opener( urllib2.HTTPCookieProcessor(cookiejar) )
response=opener.open(url)
html_doc= ' '.join(response.readlines())
 
rsssoup = BeautifulSoup(html_doc)

# test 
# list_items=rsssoup.findAll('li')
# len(list_items), list_items[:3]


3
<class 'bs4.BeautifulSoup'>
html
<class 'bs4.element.Doctype'>
<html class="no-js theme--null theme--null layout--no-sidebar" dir="ltr" lang="nl_NL">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<!--
 application.branch=hotfix/1.27.1
 application.version=1.27.1
 application.build.date=Jan 27, 2016 11:59:55 AM
 application.build.number=7
 application.git.hash=ae461494
 -->
<title>de Volkskrant</title>
<meta name="description"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" id="viewport-meta" name="viewport"/>
<meta content="on" http-equiv="cleartype"/>
<!--[if lte IE 8]>
 <link href="http://static3.persgroep.net/volkskrant/static/css/volkskrant/main-old-ie.css?r=ae461494.xyz" rel="stylesheet" />
 <![endif]-->
<!--[if gt IE 8]><!-->
<link href="http://static4.persgroep.net/volkskrant/static/css/volkskrant/main.css?r=ae461494.xyz" rel="stylesheet"/>
<!--<![endif]-->
<script src="http://static2.persgroep.net/volkskrant/stat

//anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [20]:
feeds = [link.get('href') for link in rsssoup.findAll(href=re.compile("\.xml$"))]

for link in feeds:
    response=opener.open(link)
    xml_doc= ' '.join(response.readlines())
    xmlsoup = BeautifulSoup(xml_doc, "lxml")
    name = xmlsoup.find("title").getText()
    name = name+'.html'
    with open(name, 'w') as f:
        f.write(xmlsoup.encode())
    f.close()

### JSON parsing

1. Download <http://maartenmarx.nl/teaching/ISatWork/NoteBooks/consuming-json-data-from-a-web-service.ipynb>, remove all code blocks, and turn it into a notebook again. 
2. Check that what you did is correct and you did not remove too much using a notebook viewer.
3. Now extract all code from the downloaded notebook, save it to a file, and execute it as a Python script. Does it give errors? Is it syntactically correct?

In [16]:
#import notebook
url = "http://maartenmarx.nl/teaching/ISatWork/NoteBooks/consuming-json-data-from-a-web-service.ipynb"
jsonfile= urllib2.urlopen(url)
jsonfile = jsonfile.read()  # The jsonfile as a Python string
json_as_python_object = json.loads(jsonfile) # The josnfile transformed into a Python dict
cells = json_as_python_object['worksheets'][0]['cells']
codeblocks = [cell['input'] for cell in cells if str(cell['cell_type']) == 'code']
f = open("output.py", "w")
for x in codeblocks:
    for y in x:
      f.write(str(y))
    f.write("\n")
f.close()

### PDF parsing

1. Save a wordfile as PDF, open it in Python, extract all text. Describe the differences, if any. Try the same with a two column PDF file from the web. This exercise gets more interesting if you use _difficult_ PDF. Why not try <http://wch.github.io/latexsheet/latexsheet.pdf>?

* Is the word order still as it should be?
* What about the strange characters?

# Part 2 Python recap

Download [PythonRecap2.0.ipynb](PythonRecap2.0.ipynb),  and answer all questions as asked.